In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [2]:
# Load DVC-tracked data
# Run in terminal: dvc checkout
df = pd.read_csv('data/insurance_data.csv')

C:\Users\Daniel.Temesgen\AppData\Local\Temp\ipykernel_6876\3030545956.py:3: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/insurance_data.csv')


In [7]:
print(df.columns)

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet',
       'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims', 'RiskScore', 'HasClaim', 'VehicleAge',
       'PremiumToValueRatio'],
      dtype='object')


In [4]:

# Handle missing values
# Impute TotalClaims with median
imputer = SimpleImputer(strategy='median')
df['TotalClaims'] = imputer.fit_transform(df[['TotalClaims']])
# Impute Province with mode
df['Province'].fillna(df['Province'].mode()[0], inplace=True)

# Feature engineering
# 1. HasClaim (binary for classification)
df['HasClaim'] = (df['TotalClaims'] > 0).astype(int)
# 2. VehicleAge (assume CustomValueEstimate proxies age inversely)
df['VehicleAge'] = 1 / (df['CustomValueEstimate'] / 1000)  # Higher value = newer vehicle
# 3. PremiumToValueRatio
df['PremiumToValueRatio'] = df['TotalPremium'] / df['CustomValueEstimate']



C:\Users\Daniel.Temesgen\AppData\Local\Temp\ipykernel_6876\3941814264.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Province'].fillna(df['Province'].mode()[0], inplace=True)


In [8]:
# Encode categorical variables
categorical_cols = ['Province', 'VehicleType', 'Gender', 'PostalCode', 'make']
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cols = pd.DataFrame(encoder.fit_transform(df[categorical_cols]),
                            columns=encoder.get_feature_names_out(categorical_cols))
df_encoded = pd.concat([df.drop(categorical_cols, axis=1), encoded_cols], axis=1)

# Train-test split
# Regression: Claim Severity (TotalClaims where HasClaim == 1)
severity_data = df_encoded[df_encoded['HasClaim'] == 1]
X_severity = severity_data.drop(['TotalClaims', 'HasClaim', 'TransactionMonth'], axis=1)
y_severity = severity_data['TotalClaims']
X_sev_train, X_sev_test, y_sev_train, y_sev_test = train_test_split(X_severity, y_severity, test_size=0.2, random_state=42)

# Regression: Premium Optimization (TotalPremium)
X_premium = df_encoded.drop(['TotalPremium', 'TotalClaims', 'HasClaim', 'TransactionMonth'], axis=1)
y_premium = df_encoded['TotalPremium']
X_prem_train, X_prem_test, y_prem_train, y_prem_test = train_test_split(X_premium, y_premium, test_size=0.2, random_state=42)

# Classification: Claim Probability (HasClaim)
X_claim = df_encoded.drop(['HasClaim', 'TotalClaims', 'TransactionMonth'], axis=1)
y_claim = df_encoded['HasClaim']
X_claim_train, X_claim_test, y_claim_train, y_claim_test = train_test_split(X_claim, y_claim, test_size=0.2, random_state=42)

# Commit data prep
# In terminal:
# git add notebooks/modeling_task4.ipynb
# git commit -m "Add data preparation for modeling (imputation, feature engineering, encoding)"